In [1]:
import re

def extraer_primer_numero(cadena):
    # Buscar el primer número en la cadena
    coincidencia = re.search(r'\d+', cadena)
    if coincidencia:
        return int(coincidencia.group())
    else:
        return None

def funcion_de_recompensa(cadena):
    # Diccionario para convertir números en texto a números en entero
    texto_a_numero = {
        "uno": 1,
        "dos": 2,
        "tres": 3,
        "cuatro": 4,
        "cinco": 5,
        "seis": 6,
        "siete": 7,
        "ocho": 8,
        "nueve": 9
    }

    numero_inicial = extraer_primer_numero(cadena)

    # Extraer los números en texto y sumarlos
    palabras = cadena.split()  # Ignorar "es igual a"
    suma_numeros = sum(texto_a_numero[palabra] for palabra in palabras if palabra in texto_a_numero)

    # Calcular la diferencia
    diferencia = numero_inicial - suma_numeros

    return (9-abs(diferencia))

    if diferencia == 0:
        return 1
    else:
        return 10-abs(diferencia)

funcion_de_recompensa("[INST] 10 es igual a [/INST] nueve más dos")

8

In [2]:
!pip install transformers
!pip install trl==0.11

DEPRECATION: Loading egg at /home/aurora/miniconda3/lib/python3.11/site-packages/qamar-0.1.0-py3.11-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.21.4
    Uninstalling huggingface-hub-0.21.4:
      Successfully uninstalled huggingface-hub-0.21.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
DEPRECATION: Loading egg at /home/aurora/miniconda3/lib/python3.11/site-packages/qamar-0.1.0-py3.11-linux-x86_64.egg is d

In [4]:
prompts = [
    "[INST] 1+2= [/INST]",
    "[INST] 2+2= [/INST]",
    "[INST] 3+3= [/INST]",
    "[INST] 1+1= [/INST]",
    "[INST] 0+1= [/INST]"
]
from math import log

def is_digit(x):
  return x in "1234567890"

def funcion_de_recompensa2(cadena):
    answer = cadena.split('[/INST]', 1)[1].strip()

    # if len(answer) > 8:
    #   return -log(len(answer) * 100)

    has_garbage = any(not (c.isdigit() or  c.isspace()) for c in answer)

    numbers = []
    for char in cadena:
      if is_digit(char):
        numbers.append(int(char))
    # print(numbers)
    has_number = len(numbers) > 2

    accurate = 0
    if has_number:
      a,b,c = numbers[:3]
      diff = abs(a+b-c)
      accurate = 10-diff

    result = 10
    if has_number:
      result += 50
    result += accurate * 5
    if has_garbage:
      result -= 9
    return  log(110) - log(result)

funcion_de_recompensa2("[INST] 1+1= [/INST] 2")

0.0

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from tqdm.notebook import trange
# version 0.11

# Cargar el tokenizador y el modelo preentrenado
# modelo_nombre = "C:\\Users\\NLP\\Propio\\Research\\Notebooks\\NUMBERS"
modelo_nombre = "gpt2"

# tokenizador = AutoTokenizer.from_pretrained("unsloth/mistral-7b-instruct-v0.2-bnb-4bit")
tokenizador = AutoTokenizer.from_pretrained("gpt2")
modelo = AutoModelForCausalLMWithValueHead.from_pretrained(modelo_nombre)

tokenizador.pad_token = tokenizador.eos_token

# Configurar el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelo = modelo.to(device)


# Configurar los parámetros de PPO
config_ppo = PPOConfig(
    batch_size=1,
    forward_batch_size=1,
    ppo_epochs=5,
    learning_rate=1.41e-6
)

# Inicializar el entrenador PPO
entrenador_ppo = PPOTrainer(
    config=config_ppo,
    model=modelo,
    tokenizer=tokenizador
)

# Lista de prompts
prompts = [
    "[INST] 10 es igual a [/INST]",
    "[INST] 13 es igual a [/INST]",
    "[INST] 16 es igual a [/INST]",
    "[INST] 19 es igual a [/INST]",
    "[INST] 22 es igual a [/INST]",
    "[INST] 25 es igual a [/INST]",
    "[INST] 28 es igual a [/INST]",
    "[INST] 31 es igual a [/INST]",
    "[INST] 34 es igual a [/INST]",
    "[INST] 37 es igual a [/INST]",
    "[INST] 40 es igual a [/INST]",
    "[INST] 43 es igual a [/INST]",
    "[INST] 46 es igual a [/INST]",
    "[INST] 49 es igual a [/INST]"
]

for epoch in trange(60, desc="train PPO"):
    for prompt in prompts:
        # Tokenizar el prompt y convertirlo en tensor
        entradas = tokenizador(prompt, return_tensors='pt').to(device)

        # Generar una respuesta inicial del modelo
        salida = modelo.generate(**entradas)

        respuesta = tokenizador.decode(salida[0], skip_special_tokens=True)

        # Calcular la recompensa utilizando la función personalizada
        recompensa = funcion_de_recompensa2(respuesta)


        print(f"Prompt: {prompt}")
        print(f"Respuesta: {respuesta}")
        print(f"Recompensa: {recompensa}")

        # Preparar los datos para el paso de entrenamiento PPO
        # Convertir prompt y respuesta a listas de tensores
        prompt_tensor = entradas['input_ids'][0]
        respuesta_tensor = salida[0]

        # Asegurarse de que las recompensas estén en una lista de tensores
        recompensa_tensor = torch.tensor([recompensa], dtype=torch.float).to(device)

        # Aplicar un paso de entrenamiento PPO
        entrenador_ppo.step([prompt_tensor], [respuesta_tensor], [recompensa_tensor])


train PPO:   0%|          | 0/60 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: [INST] 10 es igual a [/INST]
Respuesta: [INST] 10 es igual a [/INST] 10 es igual a [/INST] 10
Recompensa: -1


/home/aurora/miniconda3/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1309: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1760.)
  std_scores = data["scores"].std()
/home/aurora/miniconda3/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1336: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1760.)
  stats["tokens/queries_len_std"] = torch.std(query_lens).cpu().numpy().item()
/home/aurora/miniconda3/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1339: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/n

Prompt: [INST] 13 es igual a [/INST]
Respuesta: [INST] 13 es igual a [/INST] 13 es es es es es es es es
Recompensa: -4
Prompt: [INST] 16 es igual a [/INST]
Respuesta: [INST] 16 es igual a [/INST] [1] [2] [3]
Recompensa: -7


...

Prompt: [INST] 37 es igual a [/INST]
Respuesta: [INST] 37 es igual a [/INST]][/INST][/INST][/INST][/][/
Recompensa: -28


/home/aurora/miniconda3/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1313: UserWarning: KL divergence is starting to become negative: -8.79 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: [INST] 40 es igual a [/INST]
Respuesta: [INST] 40 es igual a [/INST]INST]INST:[] 40

LIST
Recompensa: -31
Prompt: [INST] 43 es igual a [/INST]
Respuesta: [INST] 43 es igual a [/INST] 43 es es(INST[INST[Thursday
Recompensa: -34


/home/aurora/miniconda3/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1313: UserWarning: KL divergence is starting to become negative: -19.38 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/home/aurora/miniconda3/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:1313: UserWarning: KL divergence is starting to become negative: -47.80 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: [INST] 46 es igual a [/INST]
Respuesta: [INST] 46 es igual a [/INST] [/ESVES] [/INSTES)[
Recompensa: -37
Prompt: [INST] 49 es igual a [/INST]
Respuesta: [INST] 49 es igual a [/INST] 49 b⑪] 49-WW
Recompensa: -40
